In [1]:
import os

import alphashape
import gmsh
import h5py
import matplotlib.pyplot as plt
import meshio
import numpy as np
import shapely
import warnings

from concavehull import concavehull
from descartes import PolygonPatch
from dolfinx import cpp, fem, io, mesh, nls, plot
from mpi4py import MPI
from petsc4py import PETSc

import commons, geometry, grapher
warnings.simplefilter('ignore')

In [2]:
markers = commons.SurfaceMarkers()
cell_types = commons.CellTypes()

### 2D Irregular Surface

In [3]:
def mesh_surface(coords, xmax=470, ymax=470):
    points = {}
    new_points = {}
    count = 0
    for row in coords:
        points[(row[0], row[1])] = count
        count += 1
    points_set = set(points.keys())
    no_full_neighbors = set()
    triangles = []
    for (x0, y0) in points_set:
        p0 = points[(x0, y0)]
        neighbors = [
            (int(x0 + 1), y0),
            (int(x0 + 1), int(y0 + 1)),
            (x0, int(y0 + 1))
        ]
        neighbor_points = [p0]
        for p in neighbors:
            v = points.get(p)
            neighbor_points.append(v)

        midpoint = (x0 + 0.5, y0 + 0.5)
        if midpoint[0] > xmax or midpoint[1] > ymax:
            continue
        points[midpoint] = count
        p2 = count
        count += 1
        for i in range(4):
            p0 = neighbor_points[i]
            if i == 3:
                p1 = neighbor_points[0]
            else:
                p1 = neighbor_points[i + 1]
            if not p0 is None and not p1 is None:
                triangles.append(
                    (p0, p1, p2)
                )
                

    return triangles, points

In [4]:
img = np.asarray(plt.imread('data/current_constriction/test11.tif')[:, :, 0], dtype=np.uint8)
img2 = img.copy()
img2[0:5, :] = 0
img2[-5:, :] = 0
img2[:, 0:5] = 0
img2[:, -5:,] = 0
coords = np.asarray(np.argwhere(img2 == 1), dtype=np.int32)

In [5]:
triangles, points = mesh_surface(coords)
cell_type = cell_types.triangle
cells = np.asarray(triangles, dtype=np.int32)
max_row_idx = max(points.values())
new_points = np.zeros((max_row_idx + 1, 2))

for k, v in points.items():
    new_points[v, :] = k
cell_data = np.zeros((len(triangles),), dtype=np.int32).tolist()
out_mesh = meshio.Mesh(points=new_points, cells={cell_type: cells}, cell_data={"name_to_read": [cell_data]})
out_mesh.write("trial.xdmf")

### 3D Surface Marking

In [6]:
# outdir = 'mesh/study_4/26-26-7.25/0.5'
# outdir = 'mesh/1/051-051-051_000-000-000'
# tria_meshname = f'{outdir}/tria.xdmf'
# tetr_meshname = f'{outdir}/tetr.xdmf'

# with io.XDMFFile(MPI.COMM_WORLD, f"{tetr_meshname}", "r") as infile3:
#     domain = infile3.read_mesh(cpp.mesh.GhostMode.none, 'Grid')
#     ct = infile3.read_meshtags(domain, name="Grid")
# domain.topology.create_connectivity(domain.topology.dim, domain.topology.dim - 1)
# with io.XDMFFile(MPI.COMM_WORLD, f"{tria_meshname}", "r") as infile3:
#     surf = infile3.read_mesh(cpp.mesh.GhostMode.none, 'Grid')
#     ft = infile3.read_meshtags(domain, name="Grid")

In [7]:
# surfaces = mesh.locate_entities_boundary(domain, 2, lambda x: np.isreal(x[1]))

In [8]:
# news = mesh.locate_entities_boundary(domain, 0, lambda x: np.isreal(x[1]))

In [9]:
# (surfaces.shape, news.shape, mesh.exterior_facet_indices(domain.topology).shape)

In [10]:
# data3 = h5py.File(tetr_meshname.replace(".xdmf", ".h5"), "r")
# data2 = h5py.File(tria_meshname.replace(".xdmf", ".h5"), "r")

### Irregular Surface Using gmsh

In [11]:
coords = np.asarray(np.argwhere(img == 1), dtype=np.int32)
xmax = 470
ymax = 470
all_points = {}
corner_points = [
    (0, 0, 0),
    (xmax, 0, 0),
    (xmax, ymax, 0),
    (0, ymax, 0)
]
other_points = []
count = 0
for row in coords:
    all_points[(row[0], row[1])] = count
    count += 1
gmsh_points = []
points_view = {int(v): (int(k[0]), int(k[1])) for k, v in all_points.items()}

In [12]:
graph = grapher.PixelGraph(points=points_view)
graph.build_graph()
graph.get_graph_pieces()
graph.n_pieces

122

#### Meshing with GMSH

In [14]:
Lx = 470
Ly = 470
Lz = 25
try:
    gmsh.finalize()
except:
    pass
gmsh.initialize()
gmsh.model.add('area')
z0_points = [
    (0, 0, 0),
    (Lx, 0, 0),
    (Lx, Ly, 0),
    (0, Ly, 0),
]
zL_points = [
    (0, 0, Lz),
    (Lx, 0, Lz),
    (Lx, Ly, Lz),
    (0, Ly, Lz),
]
points0 = []
points1 = []
lines = []

for i in range(4):
    points0.append(
        gmsh.model.occ.addPoint(*z0_points[i])
    )
for i in range(4):
    points1.append(
        gmsh.model.occ.addPoint(*zL_points[i])
    )
gmsh.model.occ.synchronize()
for i in range(-1, 3):
    lines.append(
        gmsh.model.occ.addLine(points0[i], points0[i + 1])
    )
for i in range(-1, 3):
    lines.append(
        gmsh.model.occ.addLine(points1[i], points1[i + 1])
    )
# 1 --> 5
lines.append(
    gmsh.model.occ.addLine(points0[1], points1[1])
)
# 2 --> 6
lines.append(
    gmsh.model.occ.addLine(points0[2], points1[2])
)
# 3 --> 7
lines.append(
    gmsh.model.occ.addLine(points0[3], points1[3])
)
# 0 --> 4
lines.append(
    gmsh.model.occ.addLine(points0[0], points1[0])
)
gmsh.model.occ.synchronize()

loops = []
# xy sides
loops.append(
    gmsh.model.occ.addCurveLoop(lines[:4])
)
loops.append(
    gmsh.model.occ.addCurveLoop(lines[4:8])
)
# xz sides
loops.append(
    gmsh.model.occ.addCurveLoop([lines[1]] + [lines[8]] + [lines[5]] + [lines[11]])
)
loops.append(
    gmsh.model.occ.addCurveLoop([lines[3]] + [lines[9]] + [lines[7]] + [lines[10]])
)
# yz sides
loops.append(
    gmsh.model.occ.addCurveLoop([lines[2]] + [lines[8]] + [lines[6]] + [lines[9]])
)
loops.append(
    gmsh.model.occ.addCurveLoop([lines[0]] + [lines[11]] + [lines[4]] + [lines[10]])
)
gmsh.model.occ.synchronize()
box_surfaces = []
for i in range(6):
    box_surfaces.append(
        gmsh.model.occ.addPlaneSurface((loops[i], ))
    )
gmsh.model.occ.synchronize()

side_loops = []
insulated = []
right = []
left = []

for p in graph.pieces:
    if len(p) < 50:
        continue
    arr = []
    for c in p:
        arr.append(points_view[c])
    alpha_shape = alphashape.alphashape(arr, 0.25)
    exterior = alpha_shape.exterior
    hull = []
    for c in exterior.coords:
        hull.append((c[0], c[1]))
    hull_arr = np.asarray(hull)
    hull_points = []
    for pp in hull[:-1]:
        hull_points.append(
            gmsh.model.occ.addPoint(int(pp[0]), int(pp[1]), 0)
        )
    gmsh.model.occ.synchronize()
    hull_lines = []
    for i in range(-1, len(hull_points) - 1):
        hull_lines.append(
            gmsh.model.occ.addLine(hull_points[i], hull_points[i + 1])
        )
    gmsh.model.occ.synchronize()
    hull_loop = gmsh.model.occ.addCurveLoop(hull_lines)
    side_loops.append(hull_loop)

surf_pieces = []
for sloop in side_loops:
    surf = gmsh.model.occ.addPlaneSurface((sloop, ))
    gmsh.model.occ.synchronize()
    surf_pieces.append(surf)

surfaces2 = gmsh.model.occ.getEntities(2)
surf_cut = gmsh.model.occ.cut([(2, 1)], surfaces2[6:], removeObject=True, removeTool=False)[0]

surf_cut2 = gmsh.model.occ.fuse(surf_cut, surfaces2[6:],  removeObject=False, removeTool=False)
print(surf_cut, surf_cut2)
gmsh.model.occ.synchronize()
surfaces = gmsh.model.occ.getEntities(2)

for surf in surfaces:
    com = gmsh.model.occ.getCenterOfMass(surf[0], surf[1])
    print(surf, com)
    if np.isclose(com[2], 0) and surf[1] <= 26:
        left.append(surf[1])
    elif np.isclose(com[2], Lz):
        right.append(surf[1])
    else:
        insulated.append(surf[1])
gmsh.model.addPhysicalGroup(2, list(range(2, 22)), markers.left_cc)
gmsh.model.addPhysicalGroup(2, right, markers.right_cc)
gmsh.model.addPhysicalGroup(2, list(range(22, 27)), markers.insulated)

gmsh.model.occ.synchronize()
print(gmsh.model.occ.getEntities(2), gmsh.model.occ.getEntities(3))
sloop = gmsh.model.occ.addSurfaceLoop(tuple([vv[1] for vv in gmsh.model.occ.getEntities(2)]))
volz = gmsh.model.occ.addVolume((sloop,))
vols = gmsh.model.occ.getEntities(3)
pgroup = gmsh.model.addPhysicalGroup(3, [vols[0][1]], 1)
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write("trial.msh")
gmsh.finalize()

[(2, 27), (2, 28), (2, 29), (2, 30), (2, 31)] ([(2, 27), (2, 32), (2, 33), (2, 34), (2, 35), (2, 36), (2, 37), (2, 38), (2, 39), (2, 40), (2, 41), (2, 42), (2, 43), (2, 44), (2, 45), (2, 46), (2, 47), (2, 48), (2, 49), (2, 50), (2, 51), (2, 52), (2, 53), (2, 54), (2, 55)], [[(2, 27)], [(2, 32)], [(2, 33)], [(2, 34)], [(2, 35)], [(2, 36)], [(2, 37)], [(2, 38)], [(2, 39)], [(2, 40)], [(2, 41)], [(2, 42)], [(2, 43)], [(2, 44)], [(2, 45)], [(2, 46)], [(2, 47)], [(2, 48)], [(2, 49)], [(2, 50)], [(2, 51)], [(2, 52)], [(2, 53)], [(2, 54)], [(2, 55)]])
(2, 2) (235.0, 235.0, 24.999999999999996)
(2, 3) (235.0, 0.0, 12.5)
(2, 4) (235.0, 470.0, 12.5)
(2, 5) (470.0, 235.0, 12.5)
(2, 6) (0.0, 235.0, 12.5)
(2, 7) (3.7796852646638053, 51.67954220314735, 0.0)
(2, 8) (24.501115416226376, 417.6146530468475, 0.0)
(2, 9) (54.33148693093301, 39.05221856788413, 0.0)
(2, 10) (114.67809138633262, 271.38329934479225, 0.0)
(2, 11) (115.27453881819079, 22.072901252836143, 0.0)
(2, 12) (140.05197132616487, 45.0591

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 16 (Line)
Info    : [ 10%] Meshing curve 17 (Line)
Info    : [ 10%] Meshing curve 18 (Line)
Info    : [ 10%] Meshing curve 19 (Line)
Info    : [ 10%] Meshing curve 20 (Line)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 22 (Line)
Info    : [ 10%] Meshing curve 23 (Line)
Info    : [ 10%] Meshing curve 24 (Line)
I

Info    : [ 20%] Meshing surface 11 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 12 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 13 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 14 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 15 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 16 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 17 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 18 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 19 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 20 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 21 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 22 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 23 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 24 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 25 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 26 (Plane, Frontal-Delaunay)
Info    

Error   : Invalid boundary mesh (overlapping facets) on surface 7 surface 36


Exception: Invalid boundary mesh (overlapping facets) on surface 7 surface 36

In [ ]:
outdir = 'mesh/study_2/470-470-100_000-000-000/'
scale_factor = [0.05, 0.05, 0.05]
msh = meshio.read("trial.msh")

tria_mesh_unscaled = geometry.create_mesh(msh, cell_types.triangle)
tria_mesh_unscaled.write(f"{outdir}" + 'tria_unscaled.xdmf')
tria_mesh_scaled = geometry.scale_mesh(tria_mesh_unscaled, cell_types.triangle, scale_factor=scale_factor)
tria_mesh_scaled.write(f"{outdir}" + 'tria.xdmf')

tetr_mesh_unscaled = geometry.create_mesh(msh, cell_types.tetra)
tetr_mesh_unscaled.write(f"{outdir}" + 'tetr_unscaled.xdmf')
tetr_mesh_scaled = geometry.scale_mesh(tetr_mesh_unscaled, cell_types.tetra, scale_factor=scale_factor)
tetr_mesh_scaled.write(f"{outdir}" + 'tetr.xdmf')